In [6]:
import os
import pandas

# 1. Data Extraction

Vamos começar nosso pipeline lendo o conjunto de dados fornecidos. Vamos aproveitar para ver o que tipo de informação esse conjunto de dados tem.

In [15]:
dataset_path = os.getenv("DATASET_PATH")
full_dataset_df = pandas.read_csv(dataset_path)
full_dataset_df

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38502,16561714,9813770,nossa senhora de perolas,1,7,Imagem Nossa Senhora Aparecida em Perolas 25cm,senhora aparecida bebe perolas,2018-04-02 19:43:07,71.200000,706.0,1,4,315,15.0,Decoração
38503,12716324,6483096,lembrancinha personalizada dia dos pais,1,27,LEMBRANCINHA - DIA DOS PAIS,lembrancinhas,2018-07-10 11:41:08,14.650000,7.0,1,26,1288,17.0,Lembrancinhas
38504,972256,4840665,copo metalizado,1,3,Copos Metalizados - Rosé,despedida bianca metalizados xvdakaw lembranci...,2018-02-17 15:53:23,11.580000,25.0,1,104,306,NaN,Lembrancinhas
38505,7291315,6420917,festa do pijama menino,1,36,Kit Festa do Pijama Meninos,festa pijama,2016-08-21 22:33:59,131.330000,0.0,1,11,55,NaN,Lembrancinhas


Para entender melhor o problema, vamos ver quais são as categorias desse conjunto de dados, e qual a distribuição dessas categorias.

In [17]:
full_dataset_df["category"].unique()
full_dataset_df.groupby("category").product_id.count()

category
Bebê                   7026
Bijuterias e Jóias      951
Decoração              8846
Lembrancinhas         17759
Outros                 1148
Papel e Cia            2777
Name: product_id, dtype: int64

# 2. Data formatting

Vamos separar nossos dados em conjunto de treino, teste e validação.

In [35]:
def split_train_test_validate(full_df):
    shuffled_dataset_df = full_df.sample(frac=1).reset_index(drop=True)
    train_cut_idx = int(len(full_df) * .6)
    validation_cut_idx = train_cut_idx + int(len(full_df) * .2)
    return (
        full_df[:train_cut_idx],
        full_df[train_cut_idx:validation_cut_idx],
        full_df[validation_cut_idx:]
    )

train_data, test_data, validate_data = split_train_test_validate(full_dataset_df)
assert(len(full_dataset_df) == len(train_data) + len(test_data) + len(validate_data))

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_data["title"])
training_title_count = count_vectorizer.transform(train_data["title"])

tf_transformer = TfidfTransformer(use_idf=False).fit(training_title_count)
training_title_tf = tf_transformer.transform(training_title_count)

In [70]:
def title_transformer(title_series):
    count_vector = count_vectorizer.transform(title_series)
    return tf_transformer.transform(count_vector)

In [62]:
from sklearn import svm
classifier = svm.SVC()
classifier.fit(training_title_tf, train_data["category"])

SVC()

In [72]:
test_title_tf = title_transformer(test_data["title"])
predicted = classifier.predict(test_title_tf)

In [74]:
import numpy as np
np.mean(predicted == test_data["category"])

0.8826126477080899